**Сложные слова**

1) нет PRED #может быть еще и как междометие, в тексте это предикатив

2) самого NPRO #либо как неопределенное местоимение (по семантике), либо как прилагательное (по склонению)

3) многие NPRO #может быть как прилагательное (склоняется так же), может быть как субстантивированное существительное

4) все-таки PRCL #из-за дефиса может быть неверно интерпретировано

5) надо PRED #может быть еще как наречие по морфологической схожести

6) любых ADJF #либо как неопределенное местоимение (по семантике), либо как прилагательное (по склонению)

7) тоже ADVB #в зависимости от контекста может быть частицей, союзом и наречием

8) других NPRO #либо как неопределенное местоимение (по семантике), либо как прилагательное (по склонению)

9) переполненном ADJF #либо как форма глагола 'переполнить', либо как прилагательное переполненный

In [50]:
from string import punctuation
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
from sklearn.metrics import accuracy_score
import stanza
from pymystem3 import Mystem
m = Mystem()
from nltk import ngrams

In [15]:
def tokenization(feedback): #токенизация, очищение от знаков препинания
  liste = []
  for i in feedback.split():
    i = i.rstrip(punctuation)
    if i.isalpha() or '-' in i:
      p = morph.parse(i.lower())[0].tag.POS
      liste.append((i.lower(), p))
  return liste

In [49]:
ref = [] #файл с размеченными трудными случаями
with open('reference.txt', encoding='utf-8') as f:
  d = f.readlines()
  for i in d:
    word = i.strip().split()[0]
    pos = i.strip().split()[1]
    ref.append(tuple((word, pos)))

In [47]:
corpus = []
with open('corpus.txt', encoding='utf-8') as f: #файл с текстом
  d = f.readlines()
  for i in d:
    corpus.append(i.strip())
results = []
for i in corpus:
  slov = tokenization(i)
  for j in slov:
    for k in ref:
      if j[0] == k[0]:
        if (j[0], accuracy_score(j, k)) not in results:
          results.append((j[0], accuracy_score(j, k)))
for i in results: #pymorphy
  print(i)

('нет', 1.0)
('самого', 0.5)
('многие', 0.5)
('все-таки', 1.0)
('тоже', 0.5)
('надо', 1.0)
('любых', 1.0)
('других', 0.5)
('переполненном', 1.0)


In [46]:
st_res = []
pos_d = {'ADJ':'ADJF', 'PART':'PRCL', 'PRON':'NPRO',
         'ADV':'ADVB', 'VERB':'VERB', 'NUM':'NUMR', 'DET':'ADJF'}
nlp = stanza.Pipeline('ru', processors='tokenize, pos')
for i in corpus:
  doc = nlp(i)
  sent_list = [sent.text for sent in doc.sentences]
  for sent in doc.sentences:
    for word in sent.words:
      for k in ref:
        if k[0] == word.text.lower():
          compare = (word.text.lower(), pos_d[word.upos])
          if (k[0], accuracy_score(compare, k)) not in st_res:
            st_res.append((k[0], accuracy_score(compare, k)))
for i in st_res: #stanza
  print(i)

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: ru (Russian):
| Processor | Package          |
--------------------------------
| tokenize  | syntagrus        |
| pos       | syntagrus_charlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Done loading processors!


('нет', 0.5)
('самого', 0.5)
('многие', 0.5)
('все-таки', 1.0)
('тоже', 0.5)
('надо', 0.5)
('любых', 1.0)
('других', 0.5)
('переполненном', 0.5)


In [48]:
myst_res = []
myst_dict = {'A':'ADJF', 'PART':'PRCL',
         'ADV':'ADVB', 'V':'VERB', 'NUM':'NUMR', 'DET':'ADJF', 'APRO':'ADJF',
             'SPRO':'NOUN'}
for i in corpus:
  analys = m.analyze(i)
  for word in analys:
    if 'analysis' in word:
      gr = word['analysis'][0]['gr']
      pos = gr.split('=')[0].split(',')[0]
      for j in ref:
        if j[0] == word['text'].lower():
          comp = (word['text'].lower(), myst_dict[pos])
          if (j[0], accuracy_score(comp, j)) not in myst_res:
              myst_res.append((j[0], accuracy_score(comp, j)))
for i in myst_res: #mystem
  print(i)

('нет', 0.5)
('самого', 0.5)
('многие', 1.0)
('все-таки', 1.0)
('тоже', 0.5)
('надо', 0.5)
('любых', 1.0)
('других', 0.5)
('переполненном', 1.0)


Из трех попробованных теггеров точнее всего определил pos pymorphy (5 совпадений)